In [1]:
%pylab inline
import sys
sys.path.append("..")
from loaders import imagenet_load
from importlib import reload
print(imagenet_load)
from conv import multigpu
from loaders.imagenet_load import orient
import conv
from conv import filter_gen, multigpu
from opt import ls
import numpy as np
from utils import misc
from utils import linalg
from utils import exputil
from numba import jit
import math
import concurrent.futures as fs
from pylab import imshow
import copy
from skimage.filters import gabor_kernel
from skimage.transform import rescale
from sklearn import metrics

Populating the interactive namespace from numpy and matplotlib
<module 'loaders.imagenet_load' from '../loaders/imagenet_load.py'>


In [2]:
NUM_CLASSES = 1000

In [3]:

def class_to_tarball_name_train(classname):
    classname = classname.strip()
    return "/home/ubuntu/imagenet-train/{0}-scaled.tar".format(classname)

def class_to_tarball_name_val(classname):
    classname = classname.strip()
    return "/home/ubuntu/imagenet-validation/val-{0}-scaled.tar".format(classname)



In [54]:
def normalize_images(images, min_divisor=1e-8, mode="CHW", feature_batch=None):
    if (mode == "HWC"):
        images = images.transpose(0,3,1,2)
    images = images.astype('float32')
    images /= 255.0

    orig_shape = images.shape
    images = images.reshape(images.shape[0], -1)
    n_images = images.shape[0]
    # Zero mean every feature
    images = images - np.mean(images, axis=1)[:,np.newaxis]
    # Normalize
    image_norms = np.linalg.norm(images, axis=1)/55.0
    # Get rid of really small norms
    image_norms[np.where(image_norms < min_divisor)] = 1
    # Make features unit norm
    images_normalized = images/image_norms[:,np.newaxis]
    images_normalized = images_normalized.reshape(orig_shape)
    if (mode == "HWC"):
        images_normalized = images_normalized.transpose(0,2,3,1)
    return images_normalized

In [5]:
%time loader_val = imagenet_load.ImagenetLoader(NUM_CLASSES, classes_path="../loaders/classes", tarball_func=class_to_tarball_name_val, mmap_loc="/dev/shm/imagenet_val", n_procs=32)
%time X_test = loader_val.load_all()

CPU times: user 200 ms, sys: 188 ms, total: 388 ms
Wall time: 3.71 s
Loaded 0 classes into mem
Loaded 100 classes into mem
Loaded 200 classes into mem
Loaded 300 classes into mem
Loaded 400 classes into mem
Loaded 500 classes into mem
Loaded 600 classes into mem
Loaded 700 classes into mem
Loaded 800 classes into mem
Loaded 900 classes into mem
CPU times: user 272 ms, sys: 184 ms, total: 456 ms
Wall time: 4.32 s


In [6]:
%time loader_train = imagenet_load.ImagenetLoader(NUM_CLASSES, classes_path="../loaders/classes", tarball_func=class_to_tarball_name_train, mmap_loc="/dev/shm/imagenet_train", n_procs=32)
%time X_train = loader_train.load_all()

CPU times: user 148 ms, sys: 284 ms, total: 432 ms
Wall time: 3.24 s
Loaded 0 classes into mem
Loaded 100 classes into mem
Loaded 200 classes into mem
Loaded 300 classes into mem
Loaded 400 classes into mem
Loaded 500 classes into mem
Loaded 600 classes into mem
Loaded 700 classes into mem
Loaded 800 classes into mem
Loaded 900 classes into mem
CPU times: user 480 ms, sys: 384 ms, total: 864 ms
Wall time: 1min 37s


In [ ]:
normalized_images = normalize_images(X_train[:4096].copy().astype('float32'))

In [ ]:
imshow(orient(normalized_images[6])[: , :, :])
plt.figure()
imshow(orient(X_train[3])[: , :, :])

In [7]:
y_train = loader_train.Y
y_test = loader_val.Y

In [8]:
num_pixels = X_train.shape[1]*X_train.shape[2]*X_train.shape[3]

In [7]:
def normalize_patches(patches, min_divisor=1e-8, zca_bias=0.1, mean_rgb=np.array([0,0,0])):
    patches /= 255.0
    n_patches = patches.shape[0]
    orig_shape = patches.shape
    
    patches = patches.reshape(patches.shape[0], -1)
    # Zero mean every feature
    patches = patches - np.mean(patches, axis=1)[:,np.newaxis]

    # Normalize
    patch_norms = np.linalg.norm(patches, axis=1)

    # Get rid of really small norms
    patch_norms[np.where(patch_norms < min_divisor)] = 1

    # Make features unit norm
    patches = patches/patch_norms[:,np.newaxis]

    data_means = np.mean(patches, axis=1)

    patchesCovMat = 1.0/n_patches * patches.T.dot(patches)

    (E,V) = np.linalg.eig(patchesCovMat)

    E += zca_bias
    sqrt_zca_eigs = np.sqrt(E)
    
    inv_sqrt_zca_eigs = np.diag(np.power(sqrt_zca_eigs, -1))
    
    global_ZCA = V.dot(inv_sqrt_zca_eigs).dot(V.T)
    
    patches_normalized = (patches).dot(global_ZCA).dot(global_ZCA.T)

    return patches_normalized.reshape(orig_shape)


@jit(nopython=True, nogil=True, cache=True)
def __grab_patches(images, patch_size=6, tot_patches=1e6, seed=0):
    np.random.seed(seed)
    tot_patches = int(tot_patches)
    im_idxs = np.random.choice(images.shape[0], tot_patches)
    idxs_x = np.random.choice(images.shape[3] - patch_size - 1, tot_patches)
    idxs_y = np.random.choice(images.shape[2] - patch_size - 1, tot_patches)
    idxs_x += int(math.ceil(patch_size/2))
    idxs_y += int(math.ceil(patch_size/2))
    patches = np.zeros((tot_patches, images.shape[1], patch_size, patch_size), dtype=np.float32)
    for i, (im_idx, idx_x, idx_y) in enumerate(zip(im_idxs, idxs_x, idxs_y)):
        out_patch = patches[i, :, :, :]
        grab_patch_from_idx(images[im_idx], idx_x, idx_y, patch_size, out_patch)
    return patches
    
@jit(nopython=True, nogil=True)
def grab_patch_from_idx(im, idx_x, idx_y, patch_size, outpatch):
    sidx_x = int(idx_x - patch_size/2)
    eidx_x = int(idx_x + patch_size/2)
    sidx_y = int(idx_y - patch_size/2)
    eidx_y = int(idx_y + patch_size/2)
    outpatch[:,:,:] = im[:, sidx_x:eidx_x, sidx_y:eidx_y,].astype(np.float32)
    return outpatch

def grab_patches(images, patch_size=6, tot_patches=5e5, seed=0, max_threads=50):
    idxs = misc.chunk_idxs(images.shape[0], max_threads)
    tot_patches = int(tot_patches)
    patches_per_thread = int(tot_patches/max_threads)
    with fs.ThreadPoolExecutor(max_workers=max_threads) as executor:
        futures = []
        for i,(sidx, eidx) in enumerate(idxs):
            futures.append(executor.submit(__grab_patches, images[sidx:eidx, :], 
                                           patch_size=patch_size,
                                           tot_patches=patches_per_thread,
                                           seed=seed+i
                                          ))
            
        results = np.vstack(list(map(lambda x: x.result(), futures)))
    return results 


In [12]:
idxs = np.random.choice(X_train.shape[0], int(1e5), replace=False)

In [13]:
X_train_subsample = X_train[idxs].astype('float32')

In [15]:
%time X_train_subsample_normalized = normalize_images(X_train_subsample)

CPU times: user 1min 6s, sys: 16min 24s, total: 17min 31s
Wall time: 17min 31s


In [16]:
patch_size = 12
%time patches = grab_patches(X_train_subsample_normalized, patch_size=patch_size, seed=12)

CPU times: user 6.87 s, sys: 4.13 s, total: 11 s
Wall time: 2.53 s


In [ ]:
patches.shape

In [26]:
patches_normalized = normalize_patches(patches)

In [27]:
reload(filter_gen)
patches_mmap_data = np.memmap("/dev/shm/imagenet_patches", shape=patches.shape, dtype=patches.dtype, mode="w+")
np.copyto(patches_mmap_data, patches)
patches_mmap = multigpu.MmapArray(patches_mmap_data, mode="r+")

patches_normalized_mmap_data = np.memmap("/dev/shm/imagenet_patches_normalized", shape=patches.shape, dtype=patches.dtype, mode="w+")
np.copyto(patches_normalized_mmap_data, patches_normalized)
patches_normalized_mmap = multigpu.MmapArray(patches_normalized_mmap_data, mode="r+")

fg_patches = filter_gen.make_empirical_filter_gen(patches_mmap, seed=0)
fg_patches_normalized = filter_gen.make_empirical_filter_gen(patches_normalized_mmap, seed=0)

In [48]:
gpu_handler = multigpu.MultiGpuHandler(16)
gpu_handler.kill_all_gpu_processes()
gpu_handler.start_and_wait_for_gpu_init()

In [55]:
conv_args_template = \
{ 
  "num_feature_batches":1,
  "data_batch_size":8,
  "feature_batch_size":1024,
  "pool_size":110,
  "pool_type":"avg",
  "pool_stride":90,
  "patch_size":patch_size,
  "pad":0,
  "bias": 1.0,
  "conv_stride":1,
  "random_seed": 43,
  "preprocess_batch": normalize_images
}

In [31]:
X_train_mmap = multigpu.MmapArray(X_train, mode="r+")
X_test_mmap = multigpu.MmapArray(X_test, mode="r+")

mmap_out_train_shape = conv.conv_compute_output_shape(data=X_train_mmap, **conv_args_template)
X_train_lift = np.memmap("/dev/shm/X_train_lift", mode="w+", dtype="float32", shape=mmap_out_train_shape)

mmap_out_test_shape = conv.conv_compute_output_shape(data=X_test_mmap, **conv_args_template)
X_test_lift = np.memmap("/dev/shm/X_test_lift", mode="w+", dtype="float32", shape=mmap_out_test_shape)



In [ ]:
np.mean(np.var(fg_patches_normalized(1024), axis=(1,2,3)))

In [50]:
X_train_lift_mmap = multigpu.MmapArray(X_train_lift, mode="r+")
X_test_lift_mmap = multigpu.MmapArray(X_test_lift, mode="r+")


In [60]:
X_train_lift_mmap.shape

(1281167, 2048, 3, 3)

In [56]:
train_chunk_idxs = misc.chunk_idxs(X_train.shape[0], 16)
test_chunk_idxs = misc.chunk_idxs(X_test.shape[0], 16)

In [ ]:
gpu_results = []
for gpu, (sidx, eidx) in list(zip(gpu_handler.gpus, train_chunk_idxs)):
        fg_patches = filter_gen.make_empirical_filter_gen(patches_normalized_mmap, MIN_VAR_TOL=0, seed=0)
        conv_args = conv_args_template.copy()
        conv_args["filter_gen"] = fg_patches
        X_train_mmap_gpu = copy.copy(X_train_mmap)
        X_train_mmap_gpu.idxs = (sidx, eidx)
        X_train_out_gpu = copy.copy(X_train_lift_mmap)
        X_train_out_gpu.idxs = (sidx, eidx)
        conv_args['data'] = X_train_mmap_gpu
        conv_args['output'] = X_train_out_gpu
        gpu_result = gpu.submit_async(conv.conv_mmap, **conv_args)
        gpu_results.append(gpu_result)

for gpu_result in gpu_results:
    %time gpu_result.result()

In [ ]:
gpu_results = []
for gpu, (sidx, eidx) in list(zip(gpu_handler.gpus, test_chunk_idxs)):
        fg_patches = filter_gen.make_empirical_filter_gen(patches_normalized_mmap, MIN_VAR_TOL=0, seed=0)
        conv_args = conv_args_template.copy()
        conv_args["filter_gen"] = fg_patches
        X_test_mmap_gpu = copy.copy(X_test_mmap)
        X_test_mmap_gpu.idxs = (sidx, eidx)
        X_test_out_gpu = copy.copy(X_test_lift_mmap)
        X_test_out_gpu.idxs = (sidx, eidx)
        conv_args['data'] = X_test_mmap_gpu
        conv_args['output'] = X_test_out_gpu
        gpu_result = gpu.submit_async(conv.conv_mmap, **conv_args)
        gpu_results.append(gpu_result)

for gpu_result in gpu_results:
    %time gpu_result.result()

In [135]:
X_train_lift = X_train_lift_mmap.load()

X_test_lift = X_test_lift_mmap.load()

In [63]:
X_train_lift

memmap([[[[  3.44510786e-02,   2.17423756e-02,   1.64380297e-02],
         [  4.94641671e-03,   1.15441857e-02,   3.08092497e-02],
         [  1.95613038e-02,   1.59628168e-02,   1.47663075e-02]],

        [[  4.43703793e-02,   1.79534461e-02,   2.60176379e-02],
         [  2.19456665e-02,   9.93639883e-03,   2.16778666e-02],
         [  1.39740342e-02,   1.88377928e-02,   1.74578279e-02]],

        [[  1.72023967e-01,   7.57885054e-02,   4.77684438e-02],
         [  4.93752062e-02,   4.88249175e-02,   5.91570139e-02],
         [  5.83711304e-02,   5.01980968e-02,   4.56872918e-02]],

        ..., 
        [[  1.06888330e+00,   4.72396374e-01,   2.82585770e-01],
         [  2.43238837e-01,   1.98437035e-01,   5.16744196e-01],
         [  2.40680590e-01,   3.99655581e-01,   5.45146942e-01]],

        [[  7.63911545e-01,   5.88392437e-01,   2.88235873e-01],
         [  2.56452918e-01,   3.24411929e-01,   3.96513969e-01],
         [  3.64483625e-01,   3.32522362e-01,   3.22566986e-01]],



In [41]:
X_train_lift = X_train_lift.reshape(X_train_lift.shape[0], -1)

In [64]:
X_test_lift = X_test_lift.reshape(X_test_lift.shape[0], -1)

In [71]:
from distributed import sharded_matrix

In [66]:
idxs = np.load("/home/ubuntu/scramble_idxs.npy")

In [69]:
test_idxs = np.load("/home/ubuntu/test_scramble_idxs.npy")

In [52]:
import gc
for i in range(100): gc.collect()

In [72]:
X_train_lift.shape

(1281167, 18432)

In [67]:
X_train_lift = X_train_lift[idxs, :]

In [68]:
X_train_lift = X_train_lift.reshape(X_train_lift.shape[0], -1)

In [70]:

X_test_lift = X_test_lift.reshape(X_test_lift.shape[0], -1)
X_test_lift = X_test_lift[test_idxs, :]

In [ ]:
sharded_matrix.ShardedMatrix("X_train_3_3_pool_12_12_patch_overlap_stride_1_normalized", reshard=True, data=X_train_lift, n_jobs=32, bucket="vaishaalpywrenlinalg")


Sharding matrix..... of shape (1281167, 18432)
0 4096 0 18432
8192 40961638424576286721228812288   20480   81922048028672 32768163840   24576   000 0049152327684505618432   0     
184321843218432 18432184325324836864491525324869632368647372865536


18432

        40960 778244096073728
5734400045056   69632    573448601677824942086144081920 000 0184321843218432     110592122880106496983041024000   0 


61440901128192086016 65536     90112118784126976 184321843218432 18432      98304 114688126976110592102400106496 11468818432


18432
131072122880 0000 0     94208
 00
0  118784    0 00 0    00 18432184321843218432 1843218432 18432184321843218432  0



18432

18432



1843218432 
18432



131072 135168 0 18432
8192 12288 0 18432
135168 139264 0 18432
139264 143360 0 18432
143360 147456 0 18432
147456 151552 0 18432
151552 155648 0 18432
155648 159744 0 18432
159744 163840 0 16384018432 
167936 0 18432
167936 172032 0 18432
172032 176128 0 18432
176128180224  184320180224 0 18432 
0 18432
1

In [76]:
sharded_matrix.ShardedMatrix("X_test_3_3_pool_12_12_patch_overlap_stride_1_normalized", reshard=True, data=X_test_lift, n_jobs=32, bucket="vaishaalpywrenlinalg")


Sharding matrix..... of shape (50000, 18432)
49152 50000 0 18432
1638412288 16384 20480 0  2457640961843200 
   8192409608192 18432409628672  368642867212288
  045056   3276800  409600   184320  018432
 018432 18432
4505618432 
18432
18432 
20480 
32768
4915224576   0368640   18432018432
 
18432


In [136]:
np.save("/home/ubuntu/X_train_lift", X_train_lift)

In [48]:
np.save("/home/ubuntu/X_train_lift_3_3_overlapping_pool_12_12_patch_stride_1_seed_43_3_batches", X_train_lift)

In [56]:
np.save("/home/ubuntu/X_test_lift_3_3_overlapping_pool_12_12_patch_stride_1_seed_43_3_batches", X_test_lift)

In [45]:
np.save("/home/ubuntu/y_train", loader_train.Y)
np.save("/home/ubuntu/y_test", loader_val.Y)

In [99]:
np.mean(np.sum(X_train_lift == 0, axis=(1,2,3)))/(len(X_train_lift[0].ravel()))

0.003517104743067094

In [40]:
X_train_lift.shape

(1281167, 2048, 3, 3)

In [41]:
X_train_lift.shape

(1281167, 2048, 3, 3)

In [42]:
num_feats = conv_args_template['feature_batch_size']*conv_args_template['num_feature_batches']

In [43]:
X_train_lift

memmap([[[[  5.11452436e-01,   2.83445150e-01,   1.52476490e-01],
         [  1.65434599e-01,   1.38039827e-01,   2.22627014e-01],
         [  1.48740157e-01,   2.03946054e-01,   3.18364829e-01]],

        [[  3.08602661e-01,   1.93551764e-01,   1.97584108e-01],
         [  1.49593517e-01,   2.19801798e-01,   3.68664324e-01],
         [  2.25909159e-01,   2.75555372e-01,   3.35626394e-01]],

        [[  5.44904649e-01,   2.80532062e-01,   2.54204810e-01],
         [  1.84358343e-01,   2.36028582e-01,   4.99107957e-01],
         [  2.10995838e-01,   4.44624424e-01,   7.18321979e-01]],

        ..., 
        [[  5.02392387e+00,   2.54444790e+00,   1.86420810e+00],
         [  1.61589551e+00,   1.13935673e+00,   2.17454219e+00],
         [  9.84294713e-01,   1.47237277e+00,   2.72991586e+00]],

        [[  3.36375499e+00,   1.74019587e+00,   1.80671775e+00],
         [  1.35761952e+00,   1.11615324e+00,   2.27002239e+00],
         [  1.14981556e+00,   1.68327999e+00,   2.47908521e+00]],



In [ ]:
%time X_train_lift_flat = X_train_lift.reshape(X_train_lift.shape[0], -1)

In [44]:
%time X_train_lift_flat /= np.sqrt(num_feats)

ERROR:root:Line magic function `%tiem` not found.


CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 57.2 µs


In [45]:
X_train_lift_flat

(1281167, 18432)

In [48]:
%time X_test_lift_flat = X_test_lift.reshape(X_test_lift.shape[0], -1)


CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 897 µs


In [52]:
X_train_lift_flat
X_test_lift_flat

memmap([[  3.08167487e-01,   2.63784707e-01,   3.36981714e-01, ...,
          5.10616958e-01,   7.27508843e-01,   9.35552895e-01],
       [  1.30408823e-01,   2.18853250e-01,   1.06128708e-01, ...,
          1.56704748e+00,   1.33557463e+00,   9.27308619e-01],
       [  1.00115851e-01,   1.60914242e-01,   1.18430495e-01, ...,
          1.51934564e+00,   1.49153769e+00,   1.08198786e+00],
       ..., 
       [  1.92725826e-02,   3.55420373e-02,   7.82225572e-04, ...,
          1.09741785e-01,   3.82685602e-01,   1.97269127e-01],
       [  5.61306365e-02,   5.28845638e-02,   1.60695985e-02, ...,
          8.34878802e-01,   7.16028869e-01,   4.71188247e-01],
       [  1.68057736e-02,   3.99399139e-02,   3.39946970e-02, ...,
          9.58830535e-01,   1.16052115e+00,   1.71170700e+00]], dtype=float32)

In [53]:
%time XtX = X_train_lift_flat.T.dot(X_train_lift_flat)

CPU times: user 2h 1min 12s, sys: 1h 13min 28s, total: 3h 14min 41s
Wall time: 6min 9s


In [54]:
%time y_train_one_hot = np.eye(1000)[y_train.astype('int')].astype('float32')

CPU times: user 2.58 s, sys: 4.29 s, total: 6.87 s
Wall time: 6.87 s


In [55]:
%time Xty = X_train_lift_flat.T.dot(y_train_one_hot)

CPU times: user 14min 51s, sys: 9min 34s, total: 24min 26s
Wall time: 46.2 s


In [ ]:
print("HI")

In [107]:
np.save("XtX_100_classes_conv_12_3_3_5_batches", XtX)

In [130]:
np.save("Xty_100_classes_conv_12_3_3_5_batches", Xty)

In [117]:
from opt import ls

In [155]:
reload(ls)
import scipy.linalg

In [156]:
model = scipy.linalg.solve(XtX, Xty)

In [151]:
models = []
for reg in [1e-4, 1e-]:
    %time model = ls.trainAndEvaluatePrimalModel(XTrain=X_train_lift_flat, XTest=X_test_lift_flat, XTX=XtX, reg=reg, \
                                                 labelsTest=y_test,\
                                                 labelsTrain=y_train)
    models.append(model)

../opt/ls.py:32: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if (XTX == None):


(1281167, 18432)
(1281167, 1000)
CPU times: user 42min 10s, sys: 15min 47s, total: 57min 58s
Wall time: 2min 1s


In [ ]:
models = []
for reg in [1e-4, 1e-3, 1e-2]:
    %time model = ls.block_kernel_solve(XtX, Xty, lambdav=reg, eval_fn=eval_fn, block_size=1000, epochs=4)
    models.append(model)

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
(1000, 1000)
[[  422.91189575   414.49108887    92.3732605  ...,   474.63275146
    567.9855957    324.25292969]
 [  319.91534424   482.12432861  2080.07836914 ...,   192.62753296
    278.5513916    200.94657898]
 [ 1215.83532715  2620.68554688   199.47328186 ...,  1858.48632812
   2558.09985352   985.46118164]
 ..., 
 [ 1133.28588867   845.60412598   381.36199951 ...,  1253.73999023
   1395.69799805   818.61907959]
 [  366.93841553   194.88972473   170.04086304 ...,   433.60409546
    365.01922607   253.77632141]
 [  829.02282715  2924.15136719   132.71836853 ...,  1471.39477539
   1569.6730957    793.31567383]]
0.0001
solving block 0


In [158]:
%time eval_fn(model)

ValueError: shapes are not aligned

In [160]:
X_train_lift_flat.dot(model)

dtype('float32')

In [61]:
def eval_fn(model):
     y_pred_train = np.argmax(X_train_lift_flat.dot(model), axis=1)
     train_acc = metrics.accuracy_score(y_train, y_pred_train)
    
     y_pred_test = np.argmax(X_test_lift_flat.dot(model), axis=1)
     test_acc = metrics.accuracy_score(y_test, y_pred_test)
     return ("Train", train_acc, "Test", test_acc)

In [ ]:
for reg in [5e-2, 0.1]:
    %time acc = ls.block_kernel_solve(XTrain=X_train_lift_flat, XTest=X_test_lift_flat, labelsTrain=y_train.astype('int'), labelsTest=y_test.astype('int'), reg=reg, XTX=XtX)
    print(reg, acc)

In [1028]:
results_conv = {}
for gamma in [5e-2,1e-1, 1, 10, 100]:
    K_train_rbf_conv = linalg.computeRBFGramMatrix(X_train_lift, X_train_lift, gamma=gamma)
    K_test_rbf_conv = linalg.computeRBFGramMatrix(X_test_lift, X_train_lift, gamma=gamma)
    for reg in [1e-5, 1e-4, 1e-3, 1e-2, 1e-1]:
        acc = opt.trainAndEvaluateDualModel(KTrain=K_train_rbf_conv, KTest=K_test_rbf_conv, labelsTrain=y_train.astype('int'), 
                                                                      labelsTest=y_test.astype('int'),
                                                                      reg=reg)
        print(gamma, reg, acc)
        results[(gamma,reg)] = acc

Learning Dual Model
reg is 1e-05
0.05 1e-05 (0.99769230769230766, 0.69799999999999995)
Learning Dual Model
reg is 0.0001
0.05 0.0001 (0.98038461538461541, 0.70199999999999996)
Learning Dual Model
reg is 0.001
0.05 0.001 (0.85853846153846158, 0.70599999999999996)
Learning Dual Model
reg is 0.01
0.05 0.01 (0.72246153846153849, 0.68799999999999994)
Learning Dual Model
reg is 0.1
0.05 0.1 (0.6154615384615385, 0.59799999999999998)
Learning Dual Model
reg is 1e-05
0.1 1e-05 (0.99792307692307691, 0.68200000000000005)
Learning Dual Model
reg is 0.0001
0.1 0.0001 (0.99407692307692308, 0.70599999999999996)
Learning Dual Model
reg is 0.001
0.1 0.001 (0.92723076923076919, 0.70199999999999996)
Learning Dual Model
reg is 0.01
0.1 0.01 (0.77246153846153842, 0.68999999999999995)
Learning Dual Model
reg is 0.1
0.1 0.1 (0.65407692307692311, 0.61799999999999999)
Learning Dual Model
reg is 1e-05
1 1e-05 (0.998, 0.60599999999999998)
Learning Dual Model
reg is 0.0001
1 0.0001 (0.99807692307692308, 0.6139999

KeyboardInterrupt: 

In [ ]:
X_test_lift

In [34]:
K_test_rbf_pixels

array([[ 1.        ,  0.99926612,  0.99909939, ...,  0.99929579,
         0.99898554,  0.99931603],
       [ 0.99926612,  1.        ,  0.99913969, ...,  0.99927931,
         0.9989939 ,  0.99928697],
       [ 0.99909939,  0.99913969,  1.        , ...,  0.99925441,
         0.99911216,  0.99934219],
       ..., 
       [ 0.99905437,  0.99906212,  0.99914661, ...,  0.9995055 ,
         0.99951988,  0.99959997],
       [ 0.99926577,  0.99925152,  0.99916929, ...,  0.99948263,
         0.99917748,  0.9994573 ],
       [ 0.99912478,  0.99928218,  0.99900119, ...,  0.99943302,
         0.99906306,  0.99931051]])

In [30]:
X_train_flat.shape

(13000, 196608)

In [67]:
exputil.snapshot()

/bin/bash -c 'cd /home/ubuntu ; tar -cf ./picture_web_2017-04-15-15:14:12.701102.tar pictureweb --exclude-from <(find pictureweb -size +3M)'


In [892]:
im = imread("/home/ubuntu/tiny-imagenet-200/train/n01443537/images/n01443537_0.JPEG")

In [895]:
im.shape

(64, 64, 3)

In [923]:
fc7 = np.load("/home/ubuntu/imagenet-fc7/imagenet_features_alexnet_fc7.npz")

In [924]:
X_train_fc7 = fc7["X_train"]
y_train_fc7 = fc7["y_train"]

In [925]:
X_test_fc7 = np.load("/home/ubuntu/imagenet-fc7/imagenet_test_features_alexnet_fc7.npy")
y_test_fc7 = np.load("/home/ubuntu/imagenet-fc7/imagenet_test_labels.npy")

In [967]:
idxs = np.where(y_train_fc7 < 10)[0]
idxs_test = np.where(y_test_fc7 < 10)[0]
X_train_fc7_small = X_train_fc7[idxs, :]
y_train_fc7_small = y_train_fc7[idxs].astype('int')
X_test_fc7_small = X_test_fc7[idxs_test, :]
y_test_fc7_small = y_test_fc7[idxs_test].astype('int')


In [961]:
opt.trainAndEvaluatePrimalModel(X_train_fc7_small, X_test_fc7_small, y_train_fc7_small, y_test_fc7_small, reg=1000000)

X SHAPE  (12622, 4096)
Computing XTX
Done Computing XTX


(0.95642528917762637, 0.91400000000000003)

In [952]:
labels = y_train_fc7_small

In [953]:
y = np.eye(max(labels) + 1)[labels]

In [965]:
XtX = X_train_fc7_small.T.dot(X_train_fc7_small)

In [979]:
misc.rreload(softmax)
misc.rreload(opt)
from opt import softmax
softmax.softmax_gn(X_train_fc7_small, y_train_fc7_small, X_test_fc7_small, y_test_fc7_small, XtX, step_size=1)

step 1 took 0.15542125701904297
Compting train_preds 2
Iter: 0, Train Accuracy: 0.9988115987957534, Test Accuracy: 0.9
step 1 took 0.1811075210571289
Compting train_preds 2
Iter: 1, Train Accuracy: 0.9988115987957534, Test Accuracy: 0.9
step 1 took 0.17215442657470703
Compting train_preds 2
Iter: 2, Train Accuracy: 0.9988115987957534, Test Accuracy: 0.902
step 1 took 0.16565442085266113
Compting train_preds 2
Iter: 3, Train Accuracy: 0.9988908255427033, Test Accuracy: 0.9
step 1 took 0.1684567928314209
Compting train_preds 2
Iter: 4, Train Accuracy: 0.998970052289653, Test Accuracy: 0.9
step 1 took 0.17835235595703125
Compting train_preds 2
Iter: 5, Train Accuracy: 0.9992077325305023, Test Accuracy: 0.902
step 1 took 0.16594862937927246
Compting train_preds 2
Iter: 6, Train Accuracy: 0.9993661860244019, Test Accuracy: 0.902
step 1 took 0.16495776176452637
Compting train_preds 2
Iter: 7, Train Accuracy: 0.9994454127713516, Test Accuracy: 0.902
step 1 took 0.1683351993560791
Compting tra

KeyboardInterrupt: 

In [972]:
opt.softmax

AttributeError: module 'conv.opt' has no attribute 'softmax'

In [ ]:
print("HI")